In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# 分析地址数据的格式：
# col   含义
# 0     ID
# 1     运单ID
# 2     类型
# 3     省
# 4     市
# 5     区
# 6     gpsX
# 7     gpsY
# 8     主节点
# 9     顺序

SyntaxError: invalid syntax (<ipython-input-2-82ffd8de7016>, line 1)

In [3]:
address_A_path = r'data/address_A_easy.txt'
waybill_A_path = r'data/waybill_A.txt'

address_A = pd.read_csv(address_A_path,header=None)
waybill_A = pd.read_csv(waybill_A_path,header=None)
# print(address_A)
# print(waybill_A)


loadCity_A_path = r'data/loadCity_A.txt'
UnloadCity_A_path = r'data/UnloadCity_A.txt'

loadCity_A = pd.read_csv(loadCity_A_path,header=None)
UnloadCity_A = pd.read_csv(UnloadCity_A_path,header=None)

caseNumber1 = loadCity_A.index.stop
caseNumber2 = UnloadCity_A.index.stop

result_out=[]


for p in range(0,caseNumber1):
    for q in tqdm(range(0,caseNumber2)):
#         print(loadCity_A.loc[p].values[0],loadCity_A.loc[p].values[1],
#               UnloadCity_A.loc[q].values[0],UnloadCity_A.loc[q].values[1])
        
#         Pos_A_Pro=r"陕西"
#         Pos_A_City=r"咸阳"
#         Pos_B_Pro=r"陕西"
#         Pos_B_City=r"西安"
        Pos_A_Pro =loadCity_A.loc[p].values[0]
        Pos_A_City=loadCity_A.loc[p].values[1]
        Pos_B_Pro= UnloadCity_A.loc[q].values[0]
        Pos_B_City=UnloadCity_A.loc[q].values[1]

        currentId=''
        caseNumber = address_A.index.stop
        i = 0
        count = 1
        resultIds=[]
        while i<caseNumber:

            address= address_A.loc[i].values
            addressId=address[1]
            addressPro=address[3]
            addressCity=address[4]
            #print(addressId,addressPro,addressCity)

            i=i+1
            if addressId==currentId: #如果不是新的ID
                continue
            else: #如果是新的运单ID
                currentId=addressId
                if addressPro==Pos_A_Pro and addressCity==Pos_A_City:
                    #如果是目标提货点Pos_A,直接搜索下一条数据
                    address= address_A.loc[i].values
                    addressId=address[1]
                    addressPro=address[3]
                    addressCity=address[4]
                    if addressPro==Pos_B_Pro and addressCity==Pos_B_City:
                        #如果是目标卸货点Pos_B
                        #在运单数据中，记录该订单

                        resultIds.append(addressId)
                        #print(count,addressId)
                        #count=count+1

                else:#如果不是目标提货点Pos_A
                    continue

            i=i+1


        #进行核心计算
        # address= address_A.loc[i].values
        caseNumber = waybill_A.index.stop
        i = 0
        vols=[0]*19
        tons=[0]*19

        while i<caseNumber:
            waybill=waybill_A.loc[i].values

            times = waybill[2].split('-') #拿到运单开始时间分段
            year=times[0]
            month=times[1]
            currentId=waybill[0]
            vol=waybill[4]
            ton=waybill[5]
        #     print(currentId,year,month,vol,ton)

            if currentId in resultIds:
                Id=(int(year)-2018)*12+int(month)-1
                vols[Id]=vols[Id]+vol
                tons[Id]=tons[Id]+ton

            i=i+1

        # print(len(vols),vols)
        # print(len(tons),tons)
        
        result_out.append([Pos_A_Pro,
                          Pos_A_City,
                          Pos_B_Pro,
                          Pos_B_City,
                          vols,
                          tons]
                          )
        
   


100%|█████████████████████████████████████████████| 83/83 [07:02<00:00,  5.09s/it]


In [13]:
df=pd.DataFrame(result_out)
print(df)

     0   1   2     3                                                  4  \
0   陕西  咸阳  陕西    西安  [2130.0, 360.0, 5274.0, 4514.15, 5635.0, 5124....   
1   陕西  咸阳  宁夏    银川  [25.0, 0, 275.0, 398.0, 430.0, 490.0, 427.0, 3...   
2   陕西  咸阳  陕西    咸阳  [65.0, 80.0, 549.0, 273.0, 363.0, 275.0, 440.0...   
3   陕西  咸阳  重庆    重庆  [350.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35.0, 0,...   
4   陕西  咸阳  陕西    汉中  [415.0, 60.0, 241.0, 292.0, 295.0, 445.0, 220....   
..  ..  ..  ..   ...                                                ...   
78  陕西  咸阳  湖北    荆州  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
79  陕西  咸阳  新疆  石河子市  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
80  陕西  咸阳  河南    济源  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
81  陕西  咸阳  新疆    喀什  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
82  陕西  咸阳  西藏    山南  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                                    5  
0   [868.1799999999997, 404.20000000000005,

In [15]:
outputpath=r'data/result.csv'
df.to_csv(outputpath,index=False,header=False,encoding='utf_8_sig')